## Loading the model

In [ ]:
from src import PathOrganizer
import pandas as pd
import cobra
pd.set_option("display.max_rows", 500)


lmur_pathorg = PathOrganizer()
model_dir = lmur_pathorg.get_model_version_path(mo_name="LigilactobacillusMurinus")
model = lmur_pathorg.load_model(model_dir)

## Media Dictionaries

In [ ]:
kwoji_medium = [
    "EX_cpd00588_e0",
    "EX_cpd00009_e0",
    "EX_cpd00205_e0",
    "EX_cpd00067_e0",
    "EX_cpd00254_e0",
    "EX_cpd00048_e0",
    "EX_cpd00001_e0",
    "EX_cpd00030_e0",
    "EX_cpd00099_e0",
    "EX_cpd00013_e0",
    "EX_cpd00971_e0",
    "EX_cpd00029_e0",
    "EX_cpd10515_e0",
    "EX_cpd00011_e0",
    "EX_cpd00028_e0",
    "EX_cpd00322_e0",
    "EX_cpd00060_e0",
    "EX_cpd00066_e0",
    "EX_cpd00119_e0",
    "EX_cpd00053_e0",
    "EX_cpd00023_e0",
    "EX_cpd00132_e0",
    "EX_cpd00041_e0",
    "EX_cpd00069_e0",
    "EX_cpd00065_e0",
    "EX_cpd00035_e0",
    "EX_cpd00156_e0",
    "EX_cpd00051_e0",
    "EX_cpd00129_e0",
    "EX_cpd00033_e0",
    "EX_cpd00039_e0",
    "EX_cpd00161_e0",
    "EX_cpd00107_e0",
    "EX_cpd00054_e0",
    "EX_cpd00084_e0",
    "EX_cpd00311_e0",
    "EX_cpd00182_e0",
    "EX_cpd01217_e0",
    "EX_cpd00184_e0",
    "EX_cpd00092_e0",
    "EX_cpd00218_e0",
    "EX_cpd00305_e0",
    "EX_cpd00220_e0",
    "EX_cpd00393_e0",
    "EX_cpd00104_e0",
    "EX_cpd11606_e0",
    "EX_cpd01200_e0",  # palatinose
    "EX_cpd00027_e0",  # glucose
]



## All Relevant Functions

In [ ]:
def search_reactions(met_id):
    met_tgt = model.metabolites.get_by_id(met_id)

    for rxn in met_tgt.reactions:
        print(f"{rxn.id} --> {rxn.name}")
        print(rxn.build_reaction_string(use_metabolite_names=True))
        print("")
        
def set_media():
    for ex in model.exchanges:
        ex.bounds = (-10, 10)

    model.reactions.get_by_id("EX_cpd00007_e0").bounds = (0, 100) # Oxygen
    
    for i in kwoji_medium:
        i_p = model.reactions.get_by_id(i)
        i_p.bounds = (-100, 100)

def gather_media_fluxes():
    
    model.optimize()
    result = model.summary()
    
    uptake_df = result.uptake_flux.copy()
    up_met_names = {}
    
    for met in uptake_df["metabolite"]:
        
        upt_tgt = model.metabolites.get_by_id(met)
        
        up_met_names[met] = upt_tgt.name
    
    secretion_df = result.secretion_flux.copy()
    sec_met_names = {}
    
    for met in secretion_df["metabolite"]:
        
        sec_tgt = model.metabolites.get_by_id(met)
        
        sec_met_names[met] = sec_tgt.name
        
    uptake_df["met. names"] = uptake_df["metabolite"].map(up_met_names)
    
    uptake_df = uptake_df
    
    secretion_df["met. names"] = secretion_df["metabolite"].map(sec_met_names)
    
    return uptake_df,secretion_df

def find_differences(target_df):
    
    dif_df = target_df[~target_df["reaction"].isin(kwoji_medium)]
    return dif_df


## Results

In [ ]:
set_media()

In [ ]:
u,s = gather_media_fluxes()

In [ ]:
# u.loc[:,["met. names", "flux", "reaction"]]
find_differences(target_df=u)

In [ ]:
find_differences(target_df=s)

In [ ]:
model.metabolites.get_by_id("cpd11416_c0")

In [ ]:
for met in model.reactions.get_by_id("bio1").metabolites:
    print(met.id)
    print(met.name)
    print("")

In [ ]:
model.metabolites.get_by_id("cpd15668_c0")

## Code Guidelines and Whatnot

In [ ]:
model.reactions.get_by_id("rxn10196_c0")

In [ ]:
# metID_compartment = Metabolite(
#                     id=             "",
#                     name=           "",
#                     formula=        "",
#                     compartment=    "",
#                     )

# model.add_metabolites() # Add the Metabolite variable

# model.add_boundary(model.metabolites.get_by_id("desired_met"), type="exchange")

# rxn10892_c0 = Reaction(
#                     id="",
#                     name="",
#                     subsystem="",
#                     lower_bound=-1000,  # default
#                     upper_bound=1000,   # default
#                     )

# model.add_reactions([rxn10892_c0])

# met1_c0 = model.metabolites.get_by_id("")  # Met 1 c0
# met1_e0 = model.metabolites.get_by_id("")  # Met 1 e0
# met2_c0 = model.metabolites.get_by_id("")  # Met 2 c0
# met2_e0 = model.metabolites.get_by_id("")  # Met 2 e0

# target_reaction.add_metabolites(
#     {met1_e0: -1.0, met2_e0: -1.0, met1_c0: 1.0, met2_c0: 1.0}
# )